In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# Setup

In [2]:
from src.dataset import *
import time
import pandas as pd
from transformers import AutoFeatureExtractor

/home/alessandro/.local/share/virtualenvs/music-classification-89IuhjTZ/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
RES_DIR_PATH = "res"
AUDIOS_DIR_PATH = os.path.join(RES_DIR_PATH, "mp3_data")
MODELS_DIR_PATH = os.path.join(RES_DIR_PATH, "models")
DATASETS_DIR_PATH = os.path.join(RES_DIR_PATH, "datasets")

CSV_PATH = os.path.join(RES_DIR_PATH, "samples.csv")

MODEL_NAME = "facebook/wav2vec2-base"
TARGET_FEATURE = "genre"

TEST_SIZE = 0.2

In [4]:
model_id = MODEL_NAME.replace("/", "-")
run_name = f"{model_id}-{time.strftime('%Y%m%d-%H%M%S')}"

# Analysis

In [5]:
top_n = {
  "genre": 5
}
keep_features = ["genre", "category"]

csv_base_path = CSV_PATH.split(".")[0]
filtered_csv_path = "_".join([csv_base_path] + [f"{f}{n}" for f, n in top_n.items()]) + ".csv"

if os.path.exists(filtered_csv_path):
    df = pd.read_csv(filtered_csv_path)
else:
    df = pd.read_csv(CSV_PATH)
    df = filter_df(df, audios_dir_path=AUDIOS_DIR_PATH, keep_features=keep_features, top_n=top_n)
    df.to_csv(filtered_csv_path, index=False)

df.head()

,genre,category,mp3_path,id
0,Hip Hop,Trumpet,res/mp3_data/01 Hip Hop/Abandoned Brass Stabs.mp3,01_Hip_Hop_Abandoned_Brass_Stabs
1,Hip Hop,Timpani,res/mp3_data/01 Hip Hop/Abandoned Orchestral L...,01_Hip_Hop_Abandoned_Orchestral_Layers
2,Hip Hop,Electronic Beats,res/mp3_data/01 Hip Hop/Afloat Beat.mp3,01_Hip_Hop_Afloat_Beat
3,Hip Hop,Synthesizer,res/mp3_data/01 Hip Hop/Afloat Pad.mp3,01_Hip_Hop_Afloat_Pad
4,Hip Hop,Synthetic Bass,res/mp3_data/01 Hip Hop/Afloat Sub Bass.mp3,01_Hip_Hop_Afloat_Sub_Bass


# Dataset

In [6]:
encoded_dataset_path = os.path.join(DATASETS_DIR_PATH, f"encoded-{model_id}")
encoded_dataset_path

'res/datasets/encoded-facebook-wav2vec2-base'

In [7]:
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)

/home/alessandro/.local/share/virtualenvs/music-classification-89IuhjTZ/lib/python3.10/site-packages/transformers/configuration_utils.py:375: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [70]:
encoded_ds = None
if os.path.exists(encoded_dataset_path):
    ds = datasets.load_from_disk(encoded_dataset_path)
    encoded_ds = add_audio_column(ds)
else:
    ds = get_dataset(df)
    print("Splitting dataset into train and test")
    ds = ds.train_test_split(test_size=TEST_SIZE)
    ds = add_audio_column(ds)
    print("Applying preprocessing to dataset")
    encoded_ds = ds.map(get_preprocess_func(feature_extractor), batched=True)
    encoded_ds.save_to_disk(encoded_dataset_path)

encoded_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'audio_path', 'genre', 'category', 'input_values', 'audio'],
        num_rows: 21988
    })
    test: Dataset({
        features: ['id', 'audio_path', 'genre', 'category', 'input_values', 'audio'],
        num_rows: 5497
    })
})

In [71]:
from src.utils import play_random_audios

play_random_audios(encoded_ds["train"], 3, print_features=["id", "genre"])

id: Jam_Pack_1_World_Conga_08 - genre: World/Ethnic


id: Mark_Lettieri_Headed_Beyond_Bridge_Rhythm_01 - genre: Funk


id: 20_Chromium_Fray_Frenzy_Vocal_FX_02 - genre: Bass House
